In [2]:
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import yfinance as yf
import datetime
from datetime import timedelta

# Fetching Information

First we will fetch all the finantial information regarding the stock whose price we will attempt to predict. Then, we will keep only the close price for each day, and we will graph this. Lastly, we want to train out newral network with about 80% of our available data, so we will calculate how many registers this is.

In [3]:
AAPL_stock_information = yf.download('LOW', start = '1980-07-15', end = '2020-07-14') # Fetching apple information
AAPL_stock_information.sort_values('Date')
AAPL_stock_information.dropna()
print(AAPL_stock_information.head())
AAPL_stock_information.shape

[*********************100%***********************]  1 of 1 completed
            Open      High       Low     Close  Adj Close   Volume
Date                                                              
1980-07-15   0.0  0.644531  0.621094  0.621094   0.393291   406400
1980-07-16   0.0  0.636719  0.621094  0.636719   0.403185   166400
1980-07-17   0.0  0.648438  0.628906  0.644531   0.408132  1129600
1980-07-18   0.0  0.664062  0.644531  0.664062   0.420500   540800
1980-07-21   0.0  0.699219  0.664062  0.699219   0.442761   352000


(10085, 6)

In [4]:
AAPL_close_price = AAPL_stock_information.filter(['Close']) # Droppping everything that isn't close price
AAPL_close_price.plot(title = 'Historic close price for AAPL', figsize=(16,8), xlabel = 'Date', ylabel = 'Close Price USD ($)') # Graphing

In [5]:
training_data_len = math.ceil( AAPL_close_price.shape[0] *.9) # Calculates number of training registers
training_data_len

9077

# Scaling

Now we will scale oll of our closing prices on a scale of 0 to 1, inclusive. This is generally a good practice when it comes to neural networks. We will use the MinMax method, which goes as follows:

$ X_{new} = \frac{X - X_{min}}{X_{max}-X_{min}} $

For time's sake, we will use the preimplemented MinMaxScaler package of the python *sklean* library. But, before we do this, we must make one last trasformation. Up until now we have been storing our closing prices in a *pandas* dataframe, but the pre-imlemented MinMaxScaler requires an array input, probably because the operations required to find the largest and smallest numbers iun this data structure are long, therefore we will transform this *pandas* dataset into a *numpy* array, in which theses operations can be done faster.

In [6]:
array_of_closing_values = AAPL_close_price.values # Transfor the dataframe into an array

scaler = MinMaxScaler(feature_range=(0, 1)) # Instanciate a MinMaxScaler with a [0,1] range
scaled_close_prices = scaler.fit_transform(array_of_closing_values) # Pass the data to the scaler, for it to scale it :)

scaled_close_prices

array([[0.00210844],
       [0.00222241],
       [0.00227939],
       ...,
       [0.98672488],
       [1.        ],
       [0.99358131]])

# Creating the Training Data

For this problem in particular we will create 2 data sub-sets. The first one we will call independent train or $x train$ and the decond one we will call dependant train, or $y train$. The independant train will have sets of 60 days worth of information, and the dependant train will store the closing price information for the day after the ones stored in the $x train$ position with the same index.

In [7]:
train_data = scaled_close_prices[0:training_data_len , : ] # Selects from the scaled prices the number of rows defined as training_data_len
print(train_data)

# Split the data into x_train and y_train data sets
x_train=[]
y_train = []
for row in range(60, len(train_data)):
    x_train.append(train_data[row-60:row,0])
    y_train.append(train_data[row,0])
    
# Turn the x and y trains into arrays, because the neural network trainer only takes 3-dimentional arrays
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

[[0.00210844]
 [0.00222241]
 [0.00227939]
 ...
 [0.58460368]
 [0.59817057]
 [0.60152588]]


# Building the Model

In [8]:
#Build the LSTM network model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(x_train, y_train, batch_size=1, epochs=1)

9017/9017 [==============================] - 340s 38ms/step - loss: 2.0253e-04


In [9]:
test_data = scaled_close_prices[training_data_len - 60: , : ] # We declare the test data to be the last 20% of out total information

#Create the x_test and y_test data sets
x_test = []
y_test =  array_of_closing_values[training_data_len : , : ] #Get all of the rows from index 1603 to the rest and all of the columns (in this case it's only column 'Close'), so 2003 - 1603 = 400 rows of data
for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i,0])
    
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
x_test

array([[[0.56148148],
        [0.56184621],
        [0.55491688],
        ...,
        [0.58460368],
        [0.59817057],
        [0.60152588]],

       [[0.56184621],
        [0.55491688],
        [0.55440624],
        ...,
        [0.59817057],
        [0.60152588],
        [0.60254704]],

       [[0.55491688],
        [0.55440624],
        [0.55068631],
        ...,
        [0.60152588],
        [0.60254704],
        [0.59313771]],

       ...,

       [[0.72020012],
        [0.68569922],
        [0.67184056],
        ...,
        [0.98781894],
        [0.98242141],
        [0.98774604]],

       [[0.68569922],
        [0.67184056],
        [0.70583083],
        ...,
        [0.98242141],
        [0.98774604],
        [0.98672488]],

       [[0.67184056],
        [0.70583083],
        [0.69146153],
        ...,
        [0.98774604],
        [0.98672488],
        [1.        ]]])

In [10]:
predictions = model.predict(x_test) 
predictions = scaler.inverse_transform(predictions) # Undo scaling

In [11]:
#Plot/Create the data for the graph
train = AAPL_close_price[:training_data_len]
valid = AAPL_close_price[training_data_len:]
valid['Predictions'] = predictions

#Visualize the data
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

<ipython-input-11-a2bb6d1fc640>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions


In [12]:
pd.set_option('display.max_rows', 10)
valid[['Close', 'Predictions']]

Close  Predictions
Date                               
2016-07-12   82.940002    84.245567
2016-07-13   81.650002    84.834404
2016-07-14   82.129997    84.832375
2016-07-15   81.669998    84.871704
2016-07-18   81.919998    84.779060
...                ...          ...
2020-07-07  135.020004   137.659927
2020-07-08  135.750000   137.667145
2020-07-09  135.610001   137.834579
2020-07-10  137.429993   137.933105
2020-07-13  136.550003   138.527206

[1008 rows x 2 columns]

In [14]:
array_of_predictions = valid[['Predictions']].values
array_of_real_prices = valid[['Close']].values

success = 0
failure = 0

for i in range (array_of_predictions.shape[0] - 1):
    if ((array_of_predictions[i + 1] <= array_of_predictions[i]) and (array_of_real_prices[i+1] <= array_of_real_prices[i])):
        success = success + 1
    elif ((array_of_predictions[i+1] > array_of_predictions[i]) and (array_of_real_prices[i+1] > array_of_real_prices[i])):
        success = success + 1
    elif ((array_of_predictions[i+1] < array_of_predictions[i]) and (array_of_real_prices[i+1] > array_of_real_prices[i])):
        failure = failure + 1
    elif ((array_of_predictions[i+1] > array_of_predictions[i]) and (array_of_real_prices[i+1] < array_of_real_prices[i])):
        failure = failure + 1
        
print(f'Success = {success}, Failure = {failure}, Success rate = {success/(success+failure) * 100}')
array_of_real_prices

Success = 511, Failure = 492, Success rate = 50.94715852442672


array([[ 82.94000244],
       [ 81.65000153],
       [ 82.12999725],
       ...,
       [135.61000061],
       [137.42999268],
       [136.55000305]])